In [2]:
import pandas as pd
import numpy as np
import json
import csv
# import glob
# import requests
import seaborn as sns
# import matplotlib as plt
import urllib.request, json
from collections import OrderedDict, defaultdict

# from doltpy.cli import Dolt
# from doltpy.cli.read import read_pandas, read_pandas_sql

from pandas_read_xml import flatten, fully_flatten, auto_separate_tables
import pandas_read_xml as pdx


In [1]:
xmls={'1063713659': 'lifepoint_xmls/27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml',
 '1831284280': 'lifepoint_xmls/36-4850536_LOURDES_HEALTH_StandardCharges.xml',
 '1477874337': 'lifepoint_xmls/27-2451336_NORTH_ALABAMA_MEDICAL_CENTER_StandardCharges.xml',
 '1699096552': 'lifepoint_xmls/27-2451336_SHOALS_HOSPITAL_StandardCharges.xml',
 '1033160049': 'lifepoint_xmls/62-1771866_SAINT_MARYS_REGIONAL_HEALTH_SYSTEM_StandardCharges.xml'}

In [9]:
clinton=pdx.read_xml(
        pdx.read_xml_from_path(r"lifepoint_xmls\27-3633811_CLINTON_MEMORIAL_HOSPITAL_StandardCharges.xml", "utf8"),
        ["StandardCharges", "Facility"],
    )

def col_clean(data):
            """clean columsn names of pd dataframe"""

            data.columns = (
                data.columns.str.lower()
                .str.replace(" ", "_", regex=False)
                .str.replace("-", "_", regex=False)
                .str.replace("@", "", regex=False)
                .str.replace("|", "_", regex=False)
            )
            return data

def pipe_flat(df, n):
    for i in range(n):
        # print(df.columns)
        df = df.pipe(flatten)
    return df
        
clinton_flat=pipe_flat(clinton,6)
clinton_flat=col_clean(clinton_flat)
clinton_flat.columns
clinton_flat['charge_and_type']=clinton_flat['patient_type']+clinton_flat['patient_charge_type']
print(clinton_flat['charge_and_type'].value_counts())
print(clinton_flat['patient_charge_type'].value_counts())

clinton_flat=clinton_flat[clinton_flat.patient_charge_type=='HCPCS']
clinton_flat.head()

OutpatientHCPCS    1313
InpatientHCPCS      293
InpatientDRG        182
EmergencyHCPCS        6
Name: charge_and_type, dtype: int64
HCPCS    1612
DRG       182
Name: patient_charge_type, dtype: int64


,name,createddate,patient_type,patient_charge_type,patient_charge_item_code,patient_charge_item_description,patient_charge_item_qtytype,patient_charge_item_grosscharge,patient_charge_item_discountcashcharge,patient_charge_item_minnegotiatedcharge,patient_charge_item_maxnegotiatedcharge,patient_charge_item_contracts,charge_and_type
182,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,11042,DEB SUBQ TISSUE 20 SQ CM/<,Each,187.440000,112.464000,61.470000,61.470000,"{'Contract': {'@Payer': 'AETNA - (POS)', '@Cha...",InpatientHCPCS
183,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,12032,INTMD RPR S/A/T/EXT 2.6-7.5,Each,5135.310000,3081.186000,72.410000,72.410000,{'Contract': {'@Payer': 'PARAMOUNT ADVANTAGE -...,InpatientHCPCS
184,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,25609,TREAT FX RADIAL 3+ FRAG,Each,11656.040000,6993.624000,1739.190000,1739.190000,{'Contract': {'@Payer': 'PARAMOUNT ADVANTAGE -...,InpatientHCPCS
185,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,26608,TREAT METACARPAL FRACTURE,Each,8611.330000,5166.798000,1784.630000,1784.630000,{'Contract': {'@Payer': 'PARAMOUNT ADVANTAGE -...,InpatientHCPCS
186,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,29827,ARTHROSCOP ROTATOR CUFF REPR,Each,14155.960000,8493.576000,1734.520000,1734.520000,{'Contract': {'@Payer': 'PARAMOUNT ADVANTAGE -...,InpatientHCPCS


In [12]:
pipe_flat(clinton_flat,4)

,name,createddate,patient_type,patient_charge_type,patient_charge_item_code,patient_charge_item_description,patient_charge_item_qtytype,patient_charge_item_grosscharge,patient_charge_item_discountcashcharge,patient_charge_item_minnegotiatedcharge,patient_charge_item_maxnegotiatedcharge,charge_and_type,patient_charge_item_contracts|Contract|@Payer,patient_charge_item_contracts|Contract|@Charge
0,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,93010,ELECTROCARDIOGRAM REPORT,Each,16.740000,10.044000,6.250000,10.333333,InpatientHCPCS,ADVANTAGE BY BUCKEYE - (Medicaid),8.500000
1,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,93010,ELECTROCARDIOGRAM REPORT,Each,16.740000,10.044000,6.250000,10.333333,InpatientHCPCS,AETNA - (HMO),10.333333
2,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,93010,ELECTROCARDIOGRAM REPORT,Each,16.740000,10.044000,6.250000,10.333333,InpatientHCPCS,AETNA - (POS),8.500000
3,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,93010,ELECTROCARDIOGRAM REPORT,Each,16.740000,10.044000,6.250000,10.333333,InpatientHCPCS,AETNA - (PPO),8.500000
4,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Inpatient,HCPCS,93010,ELECTROCARDIOGRAM REPORT,Each,16.740000,10.044000,6.250000,10.333333,InpatientHCPCS,ANTHEM BLUE CROSS AND BLUE SHIELD - (PPO),9.160000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,J9999,Chemotherapy drug,Each,281.750000,169.050000,50.740277,50.740277,OutpatientHCPCS,CARESOURCE OHIO - (Indemnity),50.740277
9767,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,Q5001,Hospice or home hlth in home,Each,0.010000,0.006000,0.010000,0.010000,OutpatientHCPCS,HUMANA INC. - (HMO-MR),0.010000
9768,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,S9123,Nursing care in home rn,Each,102.691071,61.614643,41.000000,41.000000,OutpatientHCPCS,UNITED HEALTHCARE INSURANCE COMPANY - (POS),41.000000
9769,CLINTON MEMORIAL HOSPITAL,1/5/2021 5:38:43 PM,Outpatient,HCPCS,S9129,"Occupational therapy, in the",Each,324.000000,194.400000,97.000000,97.000000,OutpatientHCPCS,UNITED HEALTHCARE INSURANCE COMPANY - (POS),97.000000


In [17]:
total_prices=0

for npi_number, raw_xml in xmls.items():
    # count loops
    df_raw = pdx.read_xml(
            pdx.read_xml_from_path(raw_xml, "utf8"),
            ["StandardCharges", "Facility"],
        )
    
        
        
        

    # create dictionary of xml names to column names
    COLSDICT = {
        # should end up as npi instead?
        "name": "name",
        # procedures
        "patient_charge_item_code": "code",
        "patient_charge_item_description": "long_description",
        # de-identified prices, look at how we did this with BSMH
        "patient_charge_item_discountcashcharge": "self_pay",
        "patient_charge_item_grosscharge": "gross",
        "patient_charge_item_maxnegotiatedcharge": "max",
        "patient_charge_item_minnegotiatedcharge": "min",
        # payer negotiated prices
        "patient_charge_item_contracts_contract_charge": "price",
        "patient_charge_item_contracts_contract_payer": "payer",
    }

    df = df_raw.copy()
    # npi_number=1457321036
    # add column for npi number
    df["npi_number"] = npi_number

    def col_clean(data):
        """clean columsn names of pd dataframe"""

        data.columns = (
            data.columns.str.lower()
            .str.replace(" ", "_", regex=False)
            .str.replace("-", "_", regex=False)
            .str.replace("@", "", regex=False)
            .str.replace("|", "_", regex=False)
        )
        return data

    # flatten the xml tree structure to get code, description, and standard charges
    
    for i in range(6):
        # print(df.columns)
        df = df.pipe(flatten)
    
    # clean columns    
    df = col_clean(df)
    
    # TODO maybe we've done this wrong?
    df = df[df["patient_charge_type"] == "HCPCS"]
    df = df.dropna(subset=["patient_charge_item_code"])
    
    

    # define hospital name, for debugging
    name = df.name.max()
    lower_name = (
        df.name.str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("-", "_", regex=False)
        .str.replace("@", "", regex=False)
        .str.replace("|", "_", regex=False)
        .max()
    )

    # remove commas from prices and make numeric
    df[
        [
            "patient_charge_item_discountcashcharge",
            "patient_charge_item_grosscharge",
            "patient_charge_item_maxnegotiatedcharge",
            "patient_charge_item_minnegotiatedcharge",
        ]
    ] = (
        df[
            [
                "patient_charge_item_discountcashcharge",
                "patient_charge_item_grosscharge",
                "patient_charge_item_maxnegotiatedcharge",
                "patient_charge_item_minnegotiatedcharge",
            ]
        ]
        .replace({",": ""}, regex=False)
        .apply(pd.to_numeric, 1)
    )
    # rename columns
    df.rename(columns=COLSDICT, inplace=True)

    # increment goes before everything else
    incremented = df["code"].copy()

    counter = defaultdict(int)
    for k, v in incremented.items():
        counter[v] += 1
        if counter[v] > 1:
            pre = v
            post = str("," + str(counter[v] - 1))
            incremented[k] = "%s%s" % (pre, post)
    # replace code column with new incremented version
    df["code"] = incremented

    # print("\n before procedures")
    # print(df.head(1))

    # create procedures table
    procedures = df[["code", "npi_number", "long_description"]].copy()
    procedures["short_description"] = np.nan
    
    # clean
    
    # set columns in right order
    PROC_COLS = ["code", "npi_number", "short_description", "long_description"]

    procedures = procedures[PROC_COLS]

    # create procedures csv
    new_filename = "procedures\\" + lower_name + "_" + str(npi_number) + ".csv"
    procedures.to_csv(new_filename, index=False)

    # print("\n procedures")
    # print(procedures.head(1))

    print("DEBUG")
    # CHARGES
    # select only payer-non specific charge info
    CHARGE_COLS = [
        "npi_number",
        "code",
        # "short_description",
        "self_pay",
        "gross",
        "max",
        "min",
    ]
    # create df to convert deidentified stuff to prices table
    charges = df[CHARGE_COLS].copy()
    # print("DEBUG 2")
    # rename columns
    charges.rename(columns=COLSDICT, inplace=True)

    # print("DEBUG 3")
    # melt charges
    def clean_melted_charges(dataframe):
        """used specifically to conver the metled non-payer specific charges to match prices schema"""

        # do the melting
        dataframe = dataframe.melt(id_vars=["npi_number", "code"], value_name="price")
        # set marker for inpatient & outpaitent prices
        dataframe["IP_OP"] = "UNSPECIFIED"
        dataframe["payer"] = "UNSPECIFIED"
        dataframe["caveat"] = "NONE"
        dataframe["mode"] = np.NaN

        def replace_value(col, value_dict):
            for k, v in value_dict.items():
                dataframe.loc[dataframe.variable == k, col] = v

        # define melted variable name : variable new value
        payers = {
            "gross": "Gross Charge",
            "self_pay": "Discounted Cash Charge",
            "min": "De-identifiedMin",
            "max": "De-identifiedMax",
        }
        # replace payers with the above dict
        replace_value("payer", payers)

        # define melted variable name : variable new value
        modes = {
            "gross": "GROSS",
            "self_pay": "MINIMUM",
            "min": "MAXIMUM",
            "max": "EXPECTED",
        }
        # replace modes with the above dict
        replace_value("mode", modes)

        # return dataframe
        return dataframe[
            [
                "code",
                "npi_number",
                "payer",
                "IP_OP",
                "caveat",
                "mode",
                "price",
            ]
        ]

    # print("DEBUG 4")
    # call function
    new_charges = clean_melted_charges(charges)
    # print("DEBUG 5")
    # print("\n new charges")
    # print(new_charges.head(1))

    # END CHARGES
    # # flatten the xml tree structure to get payer and charge out to each row
    for i in range(4):
        # print(df.columns)
        df = df.pipe(flatten)
    df = col_clean(df)
    # rename columns
    df.rename(columns=COLSDICT, inplace=True)
    # print("\n df columns before df price")
    # print(df.head(1))

    # remove commas from prices and make numeric
    df["price"] = df["price"].replace({",": ""}, regex=False).apply(pd.to_numeric, 1)

    # drop unnecessary columns
    df.drop(
        columns=[
            # "short_description",
            "long_description",
            # "patient_charge_item_dept",
            "self_pay",
            "gross",
            "max",
            "min",
            # "patient_charge_item_note",
            "patient_charge_item_qtytype",
            "patient_charge_type",
            "createddate",
            # "disclaimer",
        ],
        inplace=True,
    )

    # add nans as caveat
    df["IP_OP"] = "UNSPECIFIED"
    df["mode"] = "EXPECTED"
    df["caveat"] = "NONE"

    prices = pd.concat(
        [
            df[["code", "npi_number", "payer", "IP_OP", "caveat", "mode", "price"]],
            new_charges,
        ]
    )
    # create prices csv
    new_filename = "prices\\" + lower_name + "_" + str(npi_number) + ".csv"
    prices.to_csv(new_filename, index=False)
    total_prices+=len(prices)

    # create payers csv
    new_filename = "payers\\" + lower_name + "_" + str(npi_number) + ".csv"
    payers = pd.DataFrame(prices.payer.unique())
    payers.columns=['payer']
    payers.to_csv(new_filename, index=False)

    print(
        "\n succesful: "
        + name
        + " - "
        + str(npi_number)
        + " price nrows: "
        + str(len(prices))
    )
   

DEBUG

 succesful: CLINTON MEMORIAL HOSPITAL - 1063713659 price nrows: 16219
DEBUG

 succesful: LOURDES HEALTH - 1831284280 price nrows: 6440
DEBUG

 succesful: NORTH ALABAMA MEDICAL CENTER - 1477874337 price nrows: 6967
DEBUG

 succesful: SHOALS HOSPITAL - 1699096552 price nrows: 6967
DEBUG

 succesful: SAINT MARYS REGIONAL HEALTH SYSTEM - 1033160049 price nrows: 9282
